# Comparing between different Sentimental Analaysis Models

Goal: To determine the best model on hand

Description:  I have a csv of headliners, and fed it to GrokAI to generate a list of sentimental Scores. This will be used as the benchmark in the comparison.

Steps outlined:
1. Setup the file "with_sentiment_100.csv" for comparison and briefly screen through the list for outliers
2. Run through the different models and run the data through them
3. Compare using SST. Low=best High=worst

In [1]:
import pandas as pd
df = pd.read_csv("data/with_sentiment_100.csv")
df.head()

ParserError: Error tokenizing data. C error: Expected 3 fields in line 14, saw 4
